In [ ]:
#### test database query script

In [9]:
import pickle

# Path to your pickle file
pkl_path = "schema_db/stringdb.pkl"  # adjust this path as needed

# Load the pickle file
with open(pkl_path, "rb") as f:
    data = pickle.load(f)

# Display the type and optionally part of the content
print(f"Loaded object type: {type(data)}")

# If it's a dictionary, preview the keys
if isinstance(data, dict):
    print("Top-level keys:")
    for key in list(data.keys())[:10]:  # limit to first 10 keys
        print(f" - {key}")
elif isinstance(data, list):
    print("First 3 elements of list:")
    for item in data[:3]:
        print(item)
else:
    print("Preview of content:")
    print(data)

Loaded object type: <class 'dict'>
Top-level keys:
 - base_urls
 - output_formats
 - endpoints
 - parameter_details


In [10]:
data

{'base_urls': {'default': 'https://string-db.org/api',
  'stable': 'https://version-12-0.string-db.org/api'},
 'output_formats': {'text': ['tsv',
   'tsv-no-header',
   'json',
   'xml',
   'psi-mi',
   'psi-mi-tab'],
  'image': ['image', 'highres_image', 'svg']},
 'endpoints': {'get_string_ids': {'url': '{base_url}/{output_format}/get_string_ids',
   'description': 'Maps common protein names, synonyms and UniProt identifiers into STRING identifiers',
   'required': ['identifiers'],
   'optional': ['species', 'echo_query', 'limit', 'caller_identity'],
   'output_formats': ['tsv', 'tsv-no-header', 'json', 'xml']},
  'network_image': {'url': '{base_url}/{output_format}/network',
   'description': 'Retrieves the network image with input proteins highlighted in color',
   'required': ['identifiers'],
   'optional': ['species',
    'add_color_nodes',
    'add_white_nodes',
    'required_score',
    'network_type',
    'network_flavor',
    'hide_node_labels',
    'hide_disconnected_nodes',


In [8]:
schema

NameError: name 'schema' is not defined

In [9]:
import os
import re
import time
import json
import pickle
import requests
import pandas as pd
import csv
from anthropic import Anthropic
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get the API key from the environment
api_key = os.getenv("ANTHROPIC_API_KEY")

if not api_key:
    raise EnvironmentError("❌ Missing Anthropic API key. Set ANTHROPIC_API_KEY as an environment variable.")
else:
    print("✅ Anthropic API key loaded successfully.")

# Define BASE_URL constant
BASE_URL = "https://version-12-0.string-db.org/api"

def query_stringdb(
    prompt=None,
    endpoint=None,
    api_key=None,
    model="claude-3-5-haiku-20241022",
    download_image=False,
    output_dir=None,
    verbose=True,
    save_json=False,
    save_csv=False,
    filename_prefix=None,
    display_results=True,
):
    """Query the STRING protein interaction database using natural language or direct endpoint."""
    
    base_url = "https://version-12-0.string-db.org/api"

    if prompt is None and endpoint is None:
        return {"error": "Either a prompt or an endpoint must be provided"}

    # Generate timestamp for filenames
    timestamp = int(time.time())
    
    # Set default output directory
    if output_dir is None:
        output_dir = "./stringdb_results"
    os.makedirs(output_dir, exist_ok=True)
    
    # Set default filename prefix
    if filename_prefix is None:
        filename_prefix = "stringdb_query"

    if prompt:
        # Load STRING schema
        schema_path = os.path.join("schema_db", "stringdb.pkl")

        # Check if schema file exists
        if not os.path.exists(schema_path):
            return {"error": f"Schema file not found: {schema_path}"}

        with open(schema_path, "rb") as f:
            stringdb_schema = pickle.load(f)

        # Claude system prompt
        
        system_template = """
        You are a protein interaction expert specialized in using the STRING database API.
        Based on the user's natural language request, determine the appropriate STRING API endpoint and parameters.
        
        STRING API BASE URL: https://string-db.org/api
        
        COMMON ENDPOINTS:
        - /json/network - Get protein interaction network (most common)
        - /json/interaction_partners - Get interaction partners for proteins
        - /json/functional_annotation - Get functional annotations
        - /json/enrichment - Get enrichment analysis
        - /json/ppi_enrichment - Get protein-protein interaction enrichment
        - /json/homology - Get homology information
        - /json/version - Get version information
        
        STRING API SCHEMA:
        {schema}
        
        Your response should be a JSON object with the following fields:
        1. "endpoint": The API endpoint path (e.g., "/json/network")
        2. "parameters": A dictionary of parameters for the query
        3. "full_url": The complete URL to query (base_url + endpoint + parameters)
        4. "description": A brief description of what the query is doing
        5. "output_format": The format of the output (json, tsv, image, svg)
        
        IMPORTANT NOTES:
        - Use the correct endpoint names: "network" NOT "network_interactions"
        - Common species IDs: 9606 (human), 10090 (mouse), 7227 (fruit fly), 4932 (yeast)
        - For protein identifiers, use either gene names (e.g., "BRCA1") or UniProt IDs (e.g., "P38398")
        - The "required_score" parameter accepts values from 0 to 1000 (higher means more stringent)
        - Always include "caller_identity=bioagentos_api" as a parameter
        - Default species should be 9606 (human) if not specified
        - Default required_score should be 400 if not specified
        
        EXAMPLES:
        - For "Get BRCA1 interactions": use endpoint "/json/network" with identifiers=BRCA1
        - For "Find partners of TP53": use endpoint "/json/interaction_partners" with identifiers=TP53
        - For "Enrichment analysis of gene list": use endpoint "/json/enrichment"
        
        
        Return ONLY the JSON object with no additional text.
        """

        # Use Claude to generate API endpoint
        claude_result = _query_claude_for_api(
            prompt=prompt,
            schema=stringdb_schema,
            system_template=system_template,
            api_key=api_key,
            model=model,
        )

        if not claude_result["success"]:
            return claude_result

        query_info = claude_result["data"]
        endpoint = query_info.get("full_url", "")
        description = query_info.get("description", "")
        output_format = query_info.get("output_format", "json")

        if not endpoint:
            return {
                "error": "Failed to generate a valid endpoint from the prompt",
                "claude_response": claude_result.get("raw_response", "No response"),
            }

    else:
        if endpoint.startswith("/"):
            endpoint = f"{base_url}{endpoint}"
        elif not endpoint.startswith("http"):
            endpoint = f"{base_url}/{endpoint.lstrip('/')}"
        description = "Direct query to STRING API"
        output_format = "json"

        if "image" in endpoint or "svg" in endpoint:
            output_format = "image"

    is_image = output_format in ["image", "highres_image", "svg"]

    if is_image:
        if download_image:
            try:
                response = requests.get(endpoint, stream=True)
                response.raise_for_status()

                endpoint_parts = endpoint.split("/")
                filename = f"string_{endpoint_parts[-2]}_{timestamp}.{output_format}"
                file_path = os.path.join(output_dir, filename)

                with open(file_path, "wb") as f:
                    for chunk in response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)

                result = {
                    "success": True,
                    "query_info": {
                        "endpoint": endpoint,
                        "description": description,
                        "output_format": output_format,
                    },
                    "result": {
                        "image_saved": True,
                        "file_path": file_path,
                        "content_type": response.headers.get("Content-Type"),
                    },
                }

                # Save metadata as JSON if requested
                if save_json:
                    json_filename = f"{filename_prefix}_metadata_{timestamp}.json"
                    json_path = os.path.join(output_dir, json_filename)
                    with open(json_path, 'w') as f:
                        json.dump(result, f, indent=2)
                    result["json_file_path"] = json_path
                    print(f"📄 Metadata saved to: {json_path}")

                return result

            except Exception as e:
                return {
                    "success": False,
                    "error": f"Error downloading image: {str(e)}",
                    "query_info": {"endpoint": endpoint, "description": description},
                }

        else:
            return {
                "success": True,
                "query_info": {
                    "endpoint": endpoint,
                    "description": description,
                    "output_format": output_format,
                },
                "result": {
                    "image_available": True,
                    "download_url": endpoint,
                    "note": "Set download_image=True to save the image",
                },
            }

    # Query the API for non-image data
    api_result = _query_rest_api(endpoint=endpoint, method="GET", description=description)

    # Process the result for saving
    if api_result.get("success") and "result" in api_result:
        result_data = api_result["result"]
        
        # Display results if requested
        if display_results:
            print("\n📋 Results Summary:")
            formatted = _format_query_results(result_data)
            if "summary" in formatted:
                for key, value in formatted["summary"].items():
                    print(f"  • {key}: {value}")
            
            # Display sample data if available
            if "sample_data" in formatted and formatted["sample_data"]:
                print("\n📝 Sample Data:")
                if isinstance(formatted["sample_data"], list) and len(formatted["sample_data"]) > 0:
                    if isinstance(formatted["sample_data"][0], dict):
                        df_sample = pd.DataFrame(formatted["sample_data"])
                        print(df_sample.head())
                    else:
                        print(formatted["sample_data"][:5])  # Show first 5 items
        
        # Save JSON if requested
        if save_json:
            json_filename = f"{filename_prefix}_{timestamp}.json"
            json_path = os.path.join(output_dir, json_filename)
            
            # Create a comprehensive JSON structure
            json_data = {
                "query_info": {
                    "endpoint": endpoint,
                    "description": description,
                    "output_format": output_format,
                    "timestamp": timestamp,
                    "query_type": "natural_language" if prompt else "direct_endpoint"
                },
                "data": result_data
            }
            
            with open(json_path, 'w') as f:
                json.dump(json_data, f, indent=2)
            
            # Add file path to api_result
            api_result["json_file_path"] = json_path
            print(f"💾 JSON saved to: {json_path}")

        # Save CSV if requested and data is tabular
        if save_csv and isinstance(result_data, list) and len(result_data) > 0:
            csv_filename = f"{filename_prefix}_{timestamp}.csv"
            csv_path = os.path.join(output_dir, csv_filename)
            
            try:
                # Use pandas for better CSV handling
                if isinstance(result_data[0], dict):
                    df = pd.DataFrame(result_data)
                    df.to_csv(csv_path, index=False)
                    print(f"📊 CSV saved to: {csv_path} ({len(df)} rows, {len(df.columns)} columns)")
                else:
                    # Fallback to basic CSV writer
                    with open(csv_path, 'w', newline='') as f:
                        writer = csv.writer(f)
                        if isinstance(result_data[0], (list, tuple)):
                            writer.writerows(result_data)
                        else:
                            writer.writerow(["value"])
                            for item in result_data:
                                writer.writerow([item])
                    print(f"📊 CSV saved to: {csv_path}")
                
                api_result["csv_file_path"] = csv_path
                
            except Exception as e:
                error_msg = f"Failed to save CSV: {str(e)}"
                api_result["csv_error"] = error_msg
                print(f"⚠️  CSV error: {error_msg}")
        
        elif save_csv and not isinstance(result_data, list):
            error_msg = "CSV format not supported for this data type (not a list)"
            api_result["csv_error"] = error_msg
            print(f"⚠️  CSV error: {error_msg}")

    # Handle verbose vs non-verbose output
    if not verbose and "success" in api_result and api_result["success"] and "result" in api_result:
        formatted_result = _format_query_results(api_result["result"])
        
        # Preserve file paths in formatted result
        if "json_file_path" in api_result:
            formatted_result["json_file_path"] = api_result["json_file_path"]
        if "csv_file_path" in api_result:
            formatted_result["csv_file_path"] = api_result["csv_file_path"]
        if "csv_error" in api_result:
            formatted_result["csv_error"] = api_result["csv_error"]
            
        return formatted_result

    print("\n" + "=" * 50)
    return api_result


def save_protein_interactions(proteins, species=9606, output_dir="./stringdb_results", score_threshold=400):
    """
    Convenience function to save protein interactions in multiple formats.
    
    Parameters
    ----------
    proteins : list or str
        List of protein identifiers or single protein
    species : int
        NCBI taxonomy ID (default: 9606 for human)
    output_dir : str
        Directory to save results
    score_threshold : int
        Minimum confidence score (0-1000)
    
    Returns
    -------
    dict : Results with file paths
    """
    if isinstance(proteins, str):
        proteins = [proteins]
    
    protein_string = ",".join(proteins)
    endpoint = f"{BASE_URL}/json/network?identifiers={protein_string}&species={species}&required_score={score_threshold}&caller_identity=notebook_convenience"
    
    return query_stringdb(
        endpoint=endpoint,
        output_dir=output_dir,
        save_json=True,
        save_csv=True,
        filename_prefix=f"protein_interactions_{'_'.join(proteins)}"
    )


def _query_claude_for_api(prompt, schema, system_template, api_key=None, model="claude-3-5-haiku-20241022"):
    """Helper function to query Claude for generating API calls based on natural language prompts."""
    api_key = api_key or os.environ.get("ANTHROPIC_API_KEY")
    if api_key is None:
        return {
            "success": False,
            "error": "No API key provided. Set ANTHROPIC_API_KEY environment variable or provide api_key parameter.",
        }

    try:
        client = Anthropic(api_key=api_key)

        if schema is not None:
            schema_json = json.dumps(schema, indent=2)
            system_prompt = system_template.format(schema=schema_json)
        else:
            system_prompt = system_template

        response = client.messages.create(
            model=model,
            system=system_prompt,
            max_tokens=1000,
            messages=[{"role": "user", "content": prompt}],
        )

        claude_text = response.content[0].text.strip()

        json_start = claude_text.find("{")
        json_end = claude_text.rfind("}") + 1

        if json_start >= 0 and json_end > json_start:
            json_text = claude_text[json_start:json_end]
            result = json.loads(json_text)
        else:
            result = json.loads(claude_text)

        return {"success": True, "data": result, "raw_response": claude_text}

    except (json.JSONDecodeError, KeyError, IndexError) as e:
        return {
            "success": False,
            "error": f"Failed to parse Claude's response: {str(e)}",
            "raw_response": claude_text if "claude_text" in locals() else "No content found",
        }
    except Exception as e:
        return {"success": False, "error": f"Error querying Claude: {str(e)}"}


def _query_rest_api(endpoint, method="GET", params=None, headers=None, json_data=None, description=None):
    """General helper function to query REST APIs with consistent error handling."""

    if headers is None:
        headers = {"Accept": "application/json"}

    if description is None:
        description = f"{method} request to {endpoint}"

    url_error = None

    try:
        if method.upper() == "GET":
            response = requests.get(endpoint, params=params, headers=headers, timeout=30)
        elif method.upper() == "POST":
            response = requests.post(endpoint, params=params, headers=headers, json=json_data, timeout=30)
        else:
            return {"error": f"Unsupported HTTP method: {method}"}

        url_error = response.text
        response.raise_for_status()

        try:
            result = response.json()
        except ValueError:
            result = {"raw_text": response.text}

        return {
            "success": True,
            "query_info": {
                "endpoint": endpoint,
                "method": method,
                "description": description,
            },
            "result": result,
        }

    except requests.exceptions.RequestException as e:
        error_msg = str(e)
        response_text = ""

        if hasattr(e, "response") and e.response:
            try:
                error_json = e.response.json()
                error_msg = (
                    error_json.get("messages") or
                    error_json.get("message") or
                    error_json.get("error") or
                    error_json.get("detail") or str(e)
                )
            except Exception:
                response_text = e.response.text

        return {
            "success": False,
            "error": f"API error: {error_msg}",
            "query_info": {
                "endpoint": endpoint,
                "method": method,
                "description": description,
            },
            "response_url_error": url_error,
            "response_text": response_text,
        }

    except Exception as e:
        return {
            "success": False,
            "error": f"Error: {str(e)}",
            "query_info": {
                "endpoint": endpoint,
                "method": method,
                "description": description,
            },
        }


def _format_query_results(result_data):
    """Format query results for display."""
    formatted = {
        "summary": {},
        "sample_data": None
    }
    
    if isinstance(result_data, list):
        formatted["summary"]["Total items"] = len(result_data)
        if len(result_data) > 0:
            formatted["sample_data"] = result_data[:3]  # Show first 3 items
            if isinstance(result_data[0], dict):
                formatted["summary"]["Fields"] = list(result_data[0].keys())
    elif isinstance(result_data, dict):
        formatted["summary"]["Type"] = "Dictionary"
        formatted["summary"]["Keys"] = list(result_data.keys())
        formatted["sample_data"] = result_data
    else:
        formatted["summary"]["Type"] = type(result_data).__name__
        formatted["sample_data"] = result_data
    
    return formatted


def run_examples():
    """Run example queries to demonstrate the functionality."""
    
    print("🧬 STRING Database Query Examples")
    print("=" * 60)
    
    # Create results directory
    os.makedirs("./stringdb_results", exist_ok=True)
    
    examples = [
        {
            "title": "Natural language query",
            "func": lambda: query_stringdb(
                prompt="Show protein interactions for BRCA1",
                save_json=True,
                output_dir="./stringdb_results",
                filename_prefix="brca1_natural"
            )
        },
    ]
    
    results = []
    for i, example in enumerate(examples, 1):
        print(f"\n{i}. {example['title']}")
        print("-" * 40)
        try:
            result = example['func']()
            results.append(result)
            
            if result.get('success'):
                print(f"✅ Success!")
                if 'json_file_path' in result:
                    print(f"   📄 JSON: {result['json_file_path']}")
                if 'csv_file_path' in result:
                    print(f"   📊 CSV: {result['csv_file_path']}")
            else:
                print(f"❌ Failed: {result.get('error', 'Unknown error')}")
                
        except Exception as e:
            print(f"❌ Exception: {str(e)}")
            results.append({"error": str(e)})
    
    print(f"\n{'='*60}")
    print("📁 Check the ./stringdb_results directory for output files!")
    print("📊 Use pd.read_csv() or pd.read_json() to load the data")
    
    return results

✅ Anthropic API key loaded successfully.


In [12]:
#results = run_examples()

# 3. Use individual functions
result = query_stringdb("Show interactions for IL12", save_json=True)


📋 Results Summary:
  • Total items: 1
  • Fields: ['stringId_A', 'stringId_B', 'preferredName_A', 'preferredName_B', 'ncbiTaxonId', 'score', 'nscore', 'fscore', 'pscore', 'ascore', 'escore', 'dscore', 'tscore']

📝 Sample Data:
             stringId_A            stringId_B preferredName_A preferredName_B  \
0  9606.ENSP00000231228  9606.ENSP00000303231           IL12B           IL12A   

  ncbiTaxonId  score  nscore  fscore  pscore  ascore  escore  dscore  tscore  
0        9606  0.999       0       0       0   0.062   0.977     0.9   0.878  
💾 JSON saved to: ./stringdb_results\stringdb_query_1752746339.json



In [13]:
result = query_stringdb("Show binding proteins with IL12", save_json=True)


📋 Results Summary:
  • Total items: 28
  • Fields: ['stringId_A', 'stringId_B', 'preferredName_A', 'preferredName_B', 'ncbiTaxonId', 'score', 'nscore', 'fscore', 'pscore', 'ascore', 'escore', 'dscore', 'tscore']

📝 Sample Data:
             stringId_A            stringId_B preferredName_A preferredName_B  \
0  9606.ENSP00000231228  9606.ENSP00000228534           IL12B           IL23A   
1  9606.ENSP00000231228  9606.ENSP00000303231           IL12B           IL12A   
2  9606.ENSP00000231228  9606.ENSP00000470788           IL12B         IL12RB1   

   ncbiTaxonId  score  nscore  fscore  pscore  ascore  escore  dscore  tscore  
0         9606  0.998       0       0       0       0   0.963    0.90   0.527  
1         9606  0.997       0       0       0       0   0.960    0.90   0.353  
2         9606  0.977       0       0       0       0   0.929    0.54   0.350  
💾 JSON saved to: ./stringdb_results\stringdb_query_1752746450.json



In [15]:
result = query_interpro("find kinase domains of MAPK11", save_json=True)

TypeError: query_interpro() got an unexpected keyword argument 'save_json'

In [18]:
def query_interpro(prompt=None, endpoint=None, api_key=None, model="claude-3-5-haiku-20241022", max_results=3):
    """
    Query the InterPro REST API using natural language or a direct endpoint.
    
    Parameters:
    prompt (str, required): Natural language query about protein domains or families
    endpoint (str, optional): Direct endpoint path or full URL (e.g., "/entry/interpro/IPR023411" 
                             or "https://www.ebi.ac.uk/interpro/api/entry/interpro/IPR023411")
    api_key (str, optional): Anthropic API key. If None, will use ANTHROPIC_API_KEY env variable
    max_results (int): Maximum number of results to return per page
    model (str): Anthropic model to use
    
    Returns:
    dict: Dictionary containing both the query information and the InterPro API results
    
    Examples:
    - Natural language: query_interpro("Find information about kinase domains in InterPro")
    - Direct endpoint: query_interpro(endpoint="/entry/interpro/IPR023411")
    """
    # Base URL for InterPro API
    base_url = "https://www.ebi.ac.uk/interpro/api"
    
    # Default parameters
    format = "json"
    
    # Ensure we have either a prompt or an endpoint
    if prompt is None and endpoint is None:
        return {"error": "Either a prompt or an endpoint must be provided"}
    
    # If using prompt, parse with Claude
    if prompt:
               
        
        schema_path = os.path.join("schema_db", "interpro.pkl")
        if not os.path.exists(schema_path):
            return {"error": f"Schema file not found: {schema_path}"}
        with open(schema_path, "rb") as f:
            interpro_schema = pickle.load(f)

        # Create system prompt template
        system_template = """
        You are a protein domain expert specialized in using the InterPro REST API.
        
        Based on the user's natural language request, determine the appropriate InterPro REST API endpoint.
        
        INTERPRO REST API SCHEMA:
        {schema}
        
        Your response should be a JSON object with the following fields:
        1. "full_url": The complete URL to query (including the base URL "https://www.ebi.ac.uk/interpro/api")
        2. "description": A brief description of what the query is doing
        
        SPECIAL NOTES:
        - Path components for data types: entry, protein, structure, set, taxonomy, proteome
        - Common sources: interpro, pfam, cdd, uniprot, pdb
        - Protein subtypes can be "reviewed" or "unreviewed"
        - For specific entries, use lowercase accessions (e.g., "ipr000001" instead of "IPR000001")
        - Endpoints can be hierarchical like "/entry/interpro/protein/uniprot/P04637"
        
        Return ONLY the JSON object with no additional text.
        """
        
        # Query Claude to generate the API call
        claude_result = _query_claude_for_api(
            prompt=prompt,
            schema=interpro_schema,
            system_template=system_template,
            api_key=api_key,
            model=model
        )
        
        if not claude_result["success"]:
            return claude_result
        
        # Extract the full URL from Claude's response
        query_info = claude_result["data"]
        endpoint = query_info.get("full_url", "")
        description = query_info.get("description", "")
        
        if not endpoint:
            return {
                "error": "Failed to generate a valid endpoint from the prompt",
                "claude_response": claude_result.get("raw_response", "No response")
            }
    else:
        # Process provided endpoint
        # If it's just a path, add the base URL
        if endpoint.startswith("/"):
            endpoint = f"{base_url}{endpoint}"
        elif not endpoint.startswith("http"):
            endpoint = f"{base_url}/{endpoint.lstrip('/')}"
        
        description = "Direct query to provided endpoint"
    
    # Add pagination parameters
    params = {"page": 1, "page_size": max_results}
    
    # Add format parameter if not json
    if format and format != "json":
        params["format"] = format
    
    # Make the API request
    api_result = _query_rest_api(
        endpoint=endpoint,
        method="GET",
        params=params,
        description=description
    )
    
    return api_result


In [19]:
result = query_interpro("find kinase domains of MAPK11")

In [20]:
result

{'success': True,
 'query_info': {'endpoint': 'https://www.ebi.ac.uk/interpro/api/entry/interpro/protein/uniprot/P53580',
  'method': 'GET',
  'description': 'Find InterPro kinase domain entries for MAPK11 (UniProtKB protein)'},
 'result': {'count': 4,
  'next': 'https://www.ebi.ac.uk/interpro/api/entry/interpro/protein/uniprot/P53580?cursor=source%3As%3Aipr002467&page=1&page_size=3',
  'previous': None,
  'results': [{'metadata': {'accession': 'IPR000994',
     'name': 'Peptidase M24',
     'source_database': 'interpro',
     'type': 'domain',
     'integrated': None,
     'member_databases': {'pfam': {'PF00557': 'Metallopeptidase family M24'}},
     'go_terms': None},
    'proteins': [{'accession': 'p53580',
      'protein_length': 274,
      'source_database': 'reviewed',
      'organism': '1111708',
      'in_alphafold': True,
      'in_bfvd': False,
      'entry_protein_locations': [{'fragments': [{'start': 31,
          'end': 265,
          'dc-status': 'CONTINUOUS'}],
        '